## 1. Import des bibliothèques et chargement des données

In [ ]:
# Import des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Style des graphiques
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("✅ Bibliothèques importées avec succès!")

In [ ]:
# Chargement des données
DATA_PATH = '../data/'

# Charger les tables principales
try:
    fact_sales = pd.read_csv(f'{DATA_PATH}Fact_Sales.csv')
    dim_customers = pd.read_csv(f'{DATA_PATH}Dim_Customers.csv')
    dim_products = pd.read_csv(f'{DATA_PATH}Dim_Products.csv')
    dim_employees = pd.read_csv(f'{DATA_PATH}Dim_Employees.csv')
    sales_by_month = pd.read_csv(f'{DATA_PATH}Sales_By_Month.csv')
    sales_by_category = pd.read_csv(f'{DATA_PATH}Sales_By_Category.csv')
    sales_by_country = pd.read_csv(f'{DATA_PATH}Sales_By_Country.csv')
    top_products = pd.read_csv(f'{DATA_PATH}Top_Products.csv')
    
    # Convertir les dates
    fact_sales['OrderDate'] = pd.to_datetime(fact_sales['OrderDate'])
    
    print("✅ Données chargées avec succès!")
    print(f"\n📊 Fact_Sales: {len(fact_sales):,} lignes")
    print(f"👥 Dim_Customers: {len(dim_customers):,} lignes")
    print(f"📦 Dim_Products: {len(dim_products):,} lignes")
    print(f"👨‍💼 Dim_Employees: {len(dim_employees):,} lignes")
except FileNotFoundError as e:
    print(f"❌ Erreur: {e}")
    print("Veuillez d'abord exécuter le script ETL (etl_northwind.py)")

## 2. Exploration des données

In [ ]:
# Aperçu de la table des ventes
print("📊 Aperçu de Fact_Sales:")
fact_sales.head()

In [ ]:
# Statistiques descriptives
print("📈 Statistiques des ventes:")
fact_sales[['UnitPrice', 'Quantity', 'Discount', 'TotalAmount']].describe()

In [ ]:
# Informations sur les types de données
print("ℹ️ Types de données - Fact_Sales:")
fact_sales.info()

## 3. Analyse des Ventes

### 3.1 KPIs Principaux

In [ ]:
# Calcul des KPIs
total_revenue = fact_sales['TotalAmount'].sum()
total_orders = fact_sales['OrderID'].nunique()
avg_order_value = total_revenue / total_orders
total_quantity = fact_sales['Quantity'].sum()
total_customers = fact_sales['CustomerID'].nunique()
total_products_sold = fact_sales['ProductID'].nunique()

print("="*60)
print("                    📊 INDICATEURS CLÉS (KPIs)")
print("="*60)
print(f"💰 Chiffre d'affaires total:     ${total_revenue:,.2f}")
print(f"📋 Nombre de commandes:          {total_orders:,}")
print(f"💵 Valeur moyenne commande:      ${avg_order_value:,.2f}")
print(f"📦 Quantité totale vendue:       {total_quantity:,} unités")
print(f"👥 Nombre de clients:            {total_customers:,}")
print(f"🛒 Produits différents vendus:   {total_products_sold:,}")
print("="*60)

### 3.2 Évolution des ventes dans le temps

In [ ]:
# Évolution mensuelle des ventes
fig = px.line(
    sales_by_month, 
    x='Month', 
    y='TotalSales',
    color='Year',
    markers=True,
    title='📈 Évolution mensuelle des ventes par année',
    labels={'Month': 'Mois', 'TotalSales': 'Ventes ($)', 'Year': 'Année'}
)
fig.update_layout(
    xaxis=dict(tickmode='linear', tick0=1, dtick=1),
    template='plotly_white'
)
fig.show()

In [ ]:
# Ventes par trimestre
sales_by_quarter = fact_sales.groupby(['Year', 'Quarter']).agg({
    'TotalAmount': 'sum',
    'OrderID': 'nunique'
}).reset_index()
sales_by_quarter['Period'] = sales_by_quarter['Year'].astype(str) + ' Q' + sales_by_quarter['Quarter'].astype(str)

fig = px.bar(
    sales_by_quarter,
    x='Period',
    y='TotalAmount',
    color='Year',
    title='📊 Ventes par trimestre',
    labels={'TotalAmount': 'Ventes ($)', 'Period': 'Période'}
)
fig.update_layout(template='plotly_white')
fig.show()

### 3.3 Analyse par catégorie de produits

In [ ]:
# Ventes par catégorie
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type':'pie'}, {'type':'bar'}]],
    subplot_titles=('Répartition des ventes par catégorie', 'Ventes par catégorie ($)')
)

# Pie chart
fig.add_trace(
    go.Pie(
        labels=sales_by_category['CategoryName'],
        values=sales_by_category['TotalSales'],
        hole=0.4,
        textinfo='percent+label'
    ),
    row=1, col=1
)

# Bar chart
sales_sorted = sales_by_category.sort_values('TotalSales', ascending=True)
fig.add_trace(
    go.Bar(
        x=sales_sorted['TotalSales'],
        y=sales_sorted['CategoryName'],
        orientation='h',
        marker_color='steelblue'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='📦 Analyse des ventes par catégorie',
    showlegend=False,
    template='plotly_white',
    height=500
)
fig.show()

### 3.4 Top 10 Produits

In [ ]:
# Top 10 produits
fig = px.bar(
    top_products.sort_values('TotalAmount', ascending=True),
    x='TotalAmount',
    y='ProductName',
    orientation='h',
    title='🏆 Top 10 Produits par chiffre d\'affaires',
    labels={'TotalAmount': 'Ventes ($)', 'ProductName': 'Produit'},
    color='TotalAmount',
    color_continuous_scale='Blues'
)
fig.update_layout(template='plotly_white', height=500)
fig.show()

## 4. Analyse des Clients

### 4.1 Répartition géographique

In [ ]:
# Ventes par pays
sales_by_country_sorted = sales_by_country.sort_values('TotalSales', ascending=False).head(15)

fig = px.bar(
    sales_by_country_sorted,
    x='Country',
    y='TotalSales',
    color='TotalSales',
    title='🌍 Ventes par pays (Top 15)',
    labels={'TotalSales': 'Ventes ($)', 'Country': 'Pays'},
    color_continuous_scale='Viridis'
)
fig.update_layout(template='plotly_white', xaxis_tickangle=-45)
fig.show()

In [ ]:
# Carte des ventes mondiales
fig = px.choropleth(
    sales_by_country,
    locations='Country',
    locationmode='country names',
    color='TotalSales',
    title='🗺️ Carte mondiale des ventes',
    color_continuous_scale='Blues',
    labels={'TotalSales': 'Ventes ($)'}
)
fig.update_layout(template='plotly_white')
fig.show()

### 4.2 Segmentation des clients

In [ ]:
# Analyse RFM simplifiée (Récence, Fréquence, Montant)
customer_analysis = fact_sales.groupby('CustomerID').agg({
    'OrderDate': 'max',  # Dernière commande
    'OrderID': 'nunique',  # Nombre de commandes
    'TotalAmount': 'sum'  # Montant total
}).reset_index()

customer_analysis.columns = ['CustomerID', 'LastOrder', 'OrderCount', 'TotalSpent']
customer_analysis = customer_analysis.merge(
    dim_customers[['CustomerID', 'CompanyName', 'Country']], 
    on='CustomerID', 
    how='left'
)

# Top 10 clients
top_customers = customer_analysis.nlargest(10, 'TotalSpent')

fig = px.bar(
    top_customers.sort_values('TotalSpent', ascending=True),
    x='TotalSpent',
    y='CompanyName',
    orientation='h',
    title='👑 Top 10 Clients par chiffre d\'affaires',
    labels={'TotalSpent': 'Total dépensé ($)', 'CompanyName': 'Client'},
    color='TotalSpent',
    color_continuous_scale='Greens'
)
fig.update_layout(template='plotly_white', height=500)
fig.show()

## 5. Analyse des Produits

### 5.1 Distribution des prix

In [ ]:
# Distribution des prix des produits
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Distribution des prix', 'Prix par catégorie')
)

# Histogramme
fig.add_trace(
    go.Histogram(
        x=dim_products['UnitPrice'],
        nbinsx=20,
        marker_color='steelblue',
        name='Prix'
    ),
    row=1, col=1
)

# Box plot par catégorie
for category in dim_products['CategoryName'].dropna().unique():
    fig.add_trace(
        go.Box(
            y=dim_products[dim_products['CategoryName']==category]['UnitPrice'],
            name=category
        ),
        row=1, col=2
    )

fig.update_layout(
    title_text='💰 Analyse des prix des produits',
    template='plotly_white',
    height=500,
    showlegend=False
)
fig.update_xaxes(title_text='Prix ($)', row=1, col=1)
fig.update_yaxes(title_text='Fréquence', row=1, col=1)
fig.update_yaxes(title_text='Prix ($)', row=1, col=2)
fig.show()

### 5.2 Stock et discontinuité

In [ ]:
# Analyse du stock
stock_analysis = dim_products.groupby('CategoryName').agg({
    'UnitsInStock': 'sum',
    'UnitsOnOrder': 'sum',
    'Discontinued': 'sum',
    'ProductID': 'count'
}).reset_index()
stock_analysis.columns = ['Catégorie', 'Stock', 'En commande', 'Discontinués', 'Total produits']

print("📦 Analyse du stock par catégorie:")
stock_analysis

In [ ]:
# Visualisation du stock
fig = go.Figure(data=[
    go.Bar(name='En stock', x=stock_analysis['Catégorie'], y=stock_analysis['Stock']),
    go.Bar(name='En commande', x=stock_analysis['Catégorie'], y=stock_analysis['En commande'])
])
fig.update_layout(
    title='📊 Stock par catégorie',
    barmode='group',
    template='plotly_white',
    xaxis_tickangle=-45
)
fig.show()

## 6. Tableau de bord des KPIs

### Dashboard récapitulatif

In [ ]:
# Création du tableau de bord
fig = make_subplots(
    rows=3, cols=2,
    specs=[
        [{'type': 'indicator'}, {'type': 'indicator'}],
        [{'type': 'indicator'}, {'type': 'indicator'}],
        [{'type': 'bar', 'colspan': 2}, None]
    ],
    subplot_titles=('', '', '', '', 'Évolution des ventes mensuelles')
)

# KPI 1: Chiffre d'affaires
fig.add_trace(
    go.Indicator(
        mode='number',
        value=total_revenue,
        title={'text': '💰 Chiffre d\'affaires total'},
        number={'prefix': '$', 'valueformat': ',.0f'}
    ),
    row=1, col=1
)

# KPI 2: Nombre de commandes
fig.add_trace(
    go.Indicator(
        mode='number',
        value=total_orders,
        title={'text': '📋 Commandes'},
        number={'valueformat': ','}
    ),
    row=1, col=2
)

# KPI 3: Valeur moyenne
fig.add_trace(
    go.Indicator(
        mode='number',
        value=avg_order_value,
        title={'text': '💵 Panier moyen'},
        number={'prefix': '$', 'valueformat': ',.2f'}
    ),
    row=2, col=1
)

# KPI 4: Clients
fig.add_trace(
    go.Indicator(
        mode='number',
        value=total_customers,
        title={'text': '👥 Clients actifs'},
        number={'valueformat': ','}
    ),
    row=2, col=2
)

# Graphique d'évolution
monthly_sales = fact_sales.groupby(fact_sales['OrderDate'].dt.to_period('M'))['TotalAmount'].sum().reset_index()
monthly_sales['OrderDate'] = monthly_sales['OrderDate'].astype(str)

fig.add_trace(
    go.Bar(
        x=monthly_sales['OrderDate'],
        y=monthly_sales['TotalAmount'],
        marker_color='steelblue'
    ),
    row=3, col=1
)

fig.update_layout(
    title_text='📊 TABLEAU DE BORD - NORTHWIND ANALYTICS',
    template='plotly_white',
    height=800,
    showlegend=False
)

fig.show()

## 7. Export des graphiques

In [ ]:
# Sauvegarder les figures en images
import os

FIGURES_PATH = '../figures/'
os.makedirs(FIGURES_PATH, exist_ok=True)

# Figure 1: KPIs avec Matplotlib
fig_kpi, axes = plt.subplots(2, 2, figsize=(12, 8))
fig_kpi.suptitle('Indicateurs Clés de Performance (KPIs)', fontsize=16, fontweight='bold')

# Styling pour les KPIs
kpis = [
    ('Chiffre d\'affaires', f'${total_revenue:,.2f}', 'steelblue'),
    ('Commandes', f'{total_orders:,}', 'seagreen'),
    ('Panier moyen', f'${avg_order_value:,.2f}', 'coral'),
    ('Clients actifs', f'{total_customers:,}', 'purple')
]

for ax, (title, value, color) in zip(axes.flatten(), kpis):
    ax.text(0.5, 0.5, value, fontsize=28, ha='center', va='center', 
            fontweight='bold', color=color, transform=ax.transAxes)
    ax.text(0.5, 0.85, title, fontsize=14, ha='center', va='center',
            transform=ax.transAxes)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.patch.set_facecolor('#f0f0f0')

plt.tight_layout()
plt.savefig(f'{FIGURES_PATH}kpis_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Figure sauvegardée: {FIGURES_PATH}kpis_dashboard.png")

In [ ]:
# Figure 2: Ventes par catégorie
fig_cat, ax = plt.subplots(figsize=(10, 6))
sales_sorted = sales_by_category.sort_values('TotalSales', ascending=True)
colors = plt.cm.Blues(np.linspace(0.3, 0.9, len(sales_sorted)))
bars = ax.barh(sales_sorted['CategoryName'], sales_sorted['TotalSales'], color=colors)
ax.set_xlabel('Ventes ($)')
ax.set_title('Ventes par Catégorie de Produits', fontsize=14, fontweight='bold')

# Ajouter les valeurs
for bar, val in zip(bars, sales_sorted['TotalSales']):
    ax.text(val + 1000, bar.get_y() + bar.get_height()/2, 
            f'${val:,.0f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(f'{FIGURES_PATH}ventes_categories.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Figure sauvegardée: {FIGURES_PATH}ventes_categories.png")

In [ ]:
# Figure 3: Top 10 Pays
fig_country, ax = plt.subplots(figsize=(12, 6))
top_countries = sales_by_country.nlargest(10, 'TotalSales')
colors = plt.cm.Greens(np.linspace(0.3, 0.9, len(top_countries)))
bars = ax.bar(top_countries['Country'], top_countries['TotalSales'], color=colors)
ax.set_ylabel('Ventes ($)')
ax.set_xlabel('Pays')
ax.set_title('Top 10 Pays par Chiffre d\'Affaires', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')

# Ajouter les valeurs
for bar, val in zip(bars, top_countries['TotalSales']):
    ax.text(bar.get_x() + bar.get_width()/2, val + 1000, 
            f'${val/1000:.0f}K', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig(f'{FIGURES_PATH}top_pays.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Figure sauvegardée: {FIGURES_PATH}top_pays.png")

In [ ]:
# Figure 4: Évolution mensuelle
fig_trend, ax = plt.subplots(figsize=(14, 6))

monthly_data = fact_sales.groupby(fact_sales['OrderDate'].dt.to_period('M'))['TotalAmount'].sum()
monthly_data.index = monthly_data.index.astype(str)

ax.plot(monthly_data.index, monthly_data.values, marker='o', linewidth=2, 
        markersize=6, color='steelblue')
ax.fill_between(monthly_data.index, monthly_data.values, alpha=0.3, color='steelblue')
ax.set_xlabel('Période')
ax.set_ylabel('Ventes ($)')
ax.set_title('Évolution Mensuelle du Chiffre d\'Affaires', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{FIGURES_PATH}evolution_mensuelle.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Figure sauvegardée: {FIGURES_PATH}evolution_mensuelle.png")

## 8. Résumé et Conclusions

### Points clés de l'analyse

In [ ]:
# Générer un résumé automatique
top_category = sales_by_category.loc[sales_by_category['TotalSales'].idxmax(), 'CategoryName']
top_country_name = sales_by_country.loc[sales_by_country['TotalSales'].idxmax(), 'Country']
top_product_name = top_products.loc[top_products['TotalAmount'].idxmax(), 'ProductName']

print("="*70)
print("                    📋 RÉSUMÉ DE L'ANALYSE")
print("="*70)
print(f"""
🎯 PERFORMANCE GLOBALE:
   • Chiffre d'affaires total: ${total_revenue:,.2f}
   • Nombre total de commandes: {total_orders:,}
   • Panier moyen: ${avg_order_value:,.2f}
   
🏆 TOP PERFORMERS:
   • Meilleure catégorie: {top_category}
   • Meilleur pays: {top_country_name}
   • Produit star: {top_product_name}
   
👥 BASE CLIENT:
   • {total_customers} clients actifs
   • Présence dans {sales_by_country['Country'].nunique()} pays
   
📦 CATALOGUE:
   • {total_products_sold} produits vendus
   • {len(dim_products)} produits au catalogue
""")
print("="*70)

In [ ]:
print("\n✅ Analyse terminée avec succès!")
print("📁 Les figures ont été exportées dans le dossier 'figures/'")
print("📊 Les données transformées sont disponibles dans le dossier 'data/'")